# Lesson 5: Exercises

## Load CADETSettings

This imports all the standard libraries and provides some helper functions.

In [ ]:
%run ../utils.ipynb

## Exercise 1: Combine nonbinding tracer with binding components

From the tutorial, combine the dextran pulse with the langmuir experiment.
Then, start modifying:
- Feed concentrations
- Adsorption parameters
- Transport parameters
- Flow Rates

## Exercise 2: Multiple injections

For some processes, multiple injections onto a column in sequence.
Take the previous example and create an inlet profile with three injections. 
Try finding the best interval (cycle time) s.t. the column is used most efficiently.

***Hint:*** It might be advantageous to create a loop that handles the section times and inlet profiles.
Also strings can be modified in loops.
For example, the following snippet would create 'unit_000', 'unit_001', and 'unit_002'.
```Python3
for i in range(3):
    'unit_{0:03}'.format(i)
```

## Example 3: Load wash elute with three components

Add a second protein component to the LWE example from the tutorial lesson.
Assume that all parameters are the same as the first protein, only $k_a = 0.3~m^{3}_{MP}m^{-3}_{SP}s^{-1}$, and $\nu = 5.0$
